In [34]:
using StatsBase
using Combinatorics
using Plots

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/inference.jl")

# parameters

n = 60
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)
fk = k->(2*μ*k)^(-k)
fp = harmonicMean
Ω = z->groupSizePartition(z, fp, fk); # depends only on the size of the hyperedge

# sample from the HSBM with these parameters, restricting to hyperedges of size no more than kmax
kmax = 4
kmin = 1
H = sampleSBM(Z, ϑ, Ω; kmax=kmax, kmin = kmin)

hypergraph
  E: Dict{Integer,Dict}
  D: Dict{Integer,Integer}


In [36]:
# Create a random clustering with k clusters
include("jl/cut.jl")
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,kmax,kmin,Ω)

-898735.0771316434

In [37]:
# Quick checks on the group evaluation function
p4 = [4,0,0,0]
p3 = [3,1,0,0]
p2 = [2,2,0,0]
p1 = [2,1,1,0]
p0 = [1,1,1,1]
@show Ω(p4), Ω(p3),Ω(p2),Ω(p1),Ω(p0)
# I'm going to think about some group interactions functions where the output is ordered
# the same as the way I've ordered here, i.e. Ω(p(i+1) >= Ω(p(i)
# Ordering by majorization seems natural for clustering applications

(Ω(p4), Ω(p3), Ω(p2), Ω(p1), Ω(p0)) = (7.115745142575865e-5, 6.429801568008573e-5, 7.646365772731448e-5, 6.429801568008573e-5, 7.646365772731448e-5)


(7.115745142575865e-5, 6.429801568008573e-5, 7.646365772731448e-5, 6.429801568008573e-5, 7.646365772731448e-5)

In [42]:
@time T1 = first_term_eval(H,c,kmax,kmin,Ω)



  0.248093 seconds (205.53 k allocations: 382.865 MiB, 22.90% gc time)


-898735.0771316434

In [41]:
# Here's a faster way to calculate it...for now requires a different way to store the hypergraph and Omega
ff = p->fp(p)*fk(sum(p))
@time Om = build_omega(kmax,ff)

@time Hyp, w = hyperedge_formatting(H)

@time T2 = first_term_v2(Hyp,w,c,kmax,kmin,Om)

@show T1-T2

  0.028125 seconds (27.05 k allocations: 1.356 MiB)
  0.001410 seconds (9.51 k allocations: 413.625 KiB)
  0.054791 seconds (162.82 k allocations: 15.425 MiB, 21.05% gc time)
T1 - T2 = -5.436595529317856e-8


-5.436595529317856e-8